# Where is the Best place for new restaurant in SEOUL, Korea?

## intrduction
Mr ABC is the talented Turkish chef. He is looking for a place of his new restaurant.  
* He is looking for a place opted in his new Turkish restaurant in SEOUL, Korea  
* He want to  
    - Find place that restaurants in various categories is crowded by customers, visitors.
    - Find place that customers can access by public transportation like subway, or bus  
  
   
* Background
    - SEOUL is the biggest city in KOREA  
    - These days, the number of foreign tourists visiting Seoul is increasing every year.
    - There are a growing number of people looking for not only Korean food but also various world foods.  

* Mission 
    - Find the best place to open a new restaurant by data analysis


### Table of Contents


1. Download and Explore Dataset

2. Explore subway station in SEOUL, Korea

3. Analyze Each Station

4. Cluster Station

5. Conclusion   


In [2]:
#Import library
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

## 1. Download and Explore Dataset

* Data ( mainly from https://data.seoul.go.kr/dataList/datasetList.do )
    - Subway Station Location Data 
    - IN/OUT Population by Subway Station Data
    - Foreign Resident Count by District Area


### Data Set #1 : Subway Station Location 
For customer convenience, it is very important to access to new resturant easily by public transportantion. Nearby Subway station, it is the best option where we are looking for.

In [63]:
#get geo data for SEOUL subway station
subway_data = pd.read_csv("https://raw.githubusercontent.com/dreamania/github-example/master/subway.csv")
subway_data.dropna()
subway_data.drop(['station_number'], axis=1,inplace=True)
subway_data.drop_duplicates(subset=['statition_nm','ko_nm'], keep='first',inplace=True)

seoul_subway_data = subway_data[(subway_data.Lat>=37.413294)&(subway_data.Lat<=37.715133)&(subway_data.long>=126.734086)&(subway_data.long<=127.269311)]
seoul_subway_data.head()

,statition_nm,Lat,long,ko_nm
0,Seoul Station,37.554648,126.972559,서울역
1,City Hall,37.564718,126.977108,시청
2,Jonggak,37.570161,126.982923,종각
3,Jongno 3(sam)-ga,37.571607,126.991806,종로3가
4,Jongno 5(o)-ga,37.570926,127.001849,종로5가


### Data set #2 : IN/OUT Population by Subway Station
There are too many subway statition in SEOUL over 300.   
We need to find some idea about filter out subway station info on meaningless place(We need to find the best place of a new Turkish restaurant)  
For data analysis efficiency, we use IN/OUT Population Data by Subway Station

In [64]:
print('The dataframe has {} statition_nm.'.format(
        len(seoul_subway_data['statition_nm'].unique())
    )
)
## ISSUE  Too Many Subway Station in the Map
## We need to find some idea about filter out subway station info on meaningless place(We need to find the best place of a new Turkish restaurant)

The dataframe has 373 statition_nm.


In [96]:
#get subway station In/Out customer population info
subway_inout_data = pd.read_csv("https://raw.githubusercontent.com/dreamania/github-example/master/subway_inout.csv")
subway_inout_data.dropna()

subway_inout_data.head()


,line,Station_nm,IN_AM,OUT_AM,INOUT_R_AM,IN_LUNCH,OUT_LUNCH,INOUT_R_LUNCH,IN_DINNER,OUT_DINNER,INOUT_R_DINNER,OUT_AM_IN_DINNER_RATIO,IN_AM_OUT_DINNER_RATIO,INOUT_R_NIGHT,OUT_DINNER_IN_NIGHT_RATIO,IN_NIGHT,OUT_NIGHT,06-07 IN,06-07 OUT,07-08 IN,07-08 OUT,08-09 IN,08-09 OUT,09-10 IN,09-10 OUT,11-12 IN,11-12 OUT,12-13 IN,12-13 OUT,13-14 IN,13-14 OUT,16-17 IN,16-17 OUT,17-18 IN,17-18 OUT,18-19 IN,18-19 OUT,19-20 IN,19-20 OUT,20-21 IN,20-21 OUT,21-22 IN,21-22 OUT,22-23 IN,22-23 OUT,23-24 IN,23-24 OUT,00-01 IN,00-01 OUT
0,1호선,서울역,217772,586573,37.126155,296380,268834,110.246472,668638,367825,181.781554,87.726543,59.205329,174.621377,166.935949,220339,126181,15684,61286,44585,115354,89357,238583,68146,171350,92022,85227,101383,89704,102975,93903,123612,95981,167329,105879,253795,113369,144348,84947,103166,63630,101273,54836,77000,41500,37939,22083,4127,7762
1,1호선,신설동,91990,163907,56.123289,82071,73007,112.415248,174075,109591,158.840598,94.158840,83.939375,78.106193,267.216912,41012,52508,11002,9714,22990,30373,32986,78975,25012,44845,25369,24755,26835,23885,29867,24367,34959,25145,45564,30278,73275,34085,33385,26876,21851,18352,19528,17866,14189,17857,6614,13402,681,3383
2,1호선,종로5가,35665,280614,12.709630,153231,209948,72.985215,340453,127041,267.986713,82.423712,28.073614,450.300836,143.853113,88313,19612,3415,19150,6140,46654,10238,132023,15872,82787,39106,73872,49898,70185,64227,65891,76603,48341,89000,44294,127195,42015,72829,27388,51429,13344,43705,8823,32254,5928,11544,3661,810,1200
3,1호선,제기동,102950,133257,77.256729,136339,182003,74.910304,162318,109736,147.916819,82.096255,93.816068,79.249893,371.747010,29519,37248,10512,10759,26059,22594,37545,50517,28834,49387,39418,56340,45851,60016,51070,65647,64121,40776,57066,35652,55271,33751,30207,25151,19774,15182,14631,13344,9809,12590,4686,9194,393,2120
4,1호선,동대문,84950,93736,90.626867,91583,121988,75.075417,118178,124721,94.753891,79.317639,68.112026,81.178602,265.121272,47043,57950,12614,9043,19979,17282,26926,33995,25431,33416,26168,41333,30809,40599,34606,40056,34975,32769,35787,32860,35427,32845,25533,33797,21431,25219,19460,19798,16831,18679,9553,14421,1199,5052


#### Exploratory Data analysis
I found some meaningful derived features using exploratory Data analysis about subway station In/Out customer population info as below

* IN_AM : 06-10 customer count into subway station
* OUT_AM : 06-10 customer count out of subway station
* IN_LUNCH : 11-14  customer count into subway station
* OUT_LUNCH : 11-14 customer count out of subway station
* IN_DINNER : 17-20  customer count into subway station
* OUT_DINNER : 17-20 customer count out of subway station
* IN_NIGHT : 22-01  customer count into subway station
* OUT_NIGHT : 22-01 customer count out of subway station

Meaningful Ratio Features
* IN_AM_OUT_DINNER_RATIO : IN_AM / OUT_DINNER *100 (%) - if this ratio is below 100 %, the customer counts in dinner time are more than customer counts in AM(get to work) time using the subway station
* INOUT_R_NIGHT : IN_NIGHT / OUT_NIGHT *100 (%) - if this ratio is over 100 %, the customer counts who go back home after dinner in night time are more than customer counts go out the subway station who lived in nearby station
* OUT_DINNER_IN_NIGHT_RATIO : OUT_DINNER / IN_NIGHT *100 (%) - This ratio provide additional information about how much the customers who go out the station in dinner time  go back in Night time.



#### Filter out meaningless subway station Info

In [97]:
# Found meaningful derived feature thereshold
# Filter out meaningless subway station Info
fitered_seoul_subway_by_inout = subway_inout_data[(subway_inout_data['IN_AM_OUT_DINNER_RATIO']<=100.0)&(subway_inout_data['INOUT_R_NIGHT']>=100.0)&(subway_inout_data['OUT_DINNER_IN_NIGHT_RATIO']>=150.0)]
fitered_seoul_subway_by_inout.drop_duplicates(subset=['Station_nm'], keep='first',inplace=True)
df.head(100)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### merge data : the selected subway station by Population Analysis

In [99]:
##merge
df=pd.merge(seoul_subway_data,fitered_seoul_subway_by_inout, how='inner',left_on="ko_nm",right_on="Station_nm",indicator='indicator_info')
#df_both = df[(df['indicator_info']=='both')]

### Dataset #3 Foreign Resident Count by District Area
In the early days of its opening, foreign residents will probably be the main customers of the newly opened restaurant.
By analyzing the status of each distinct foreign resident, we want to find the most likely area to enter into the initial customer base.

In [91]:
## get Foreign Resident Count by District
## Foreign Resident counts have removed Korean-Chinese Foreign Resident 
foreigner_data = pd.read_csv("https://raw.githubusercontent.com/dreamania/github-example/master/foreigner.csv")
foreigner_data.head()
foreigner_data.dtypes

ko_nm                object
en_nm                object
Lat                 float64
long                float64
Foreign Resident      int64
Total                 int64
KO-Chinese            int64
dtype: object

## 2.Explore Neighborhoods in SEOUL, Korea
We want to find the best place to open a new restaurant by marking the location of the subway station selected on the map and the foreign residents of each distinction with markers.
* BLUE : selected subway station by crowed population in dinner time  
* RED  : Foreign Resident Count (The size of the circle is proportional to the number of foreign resident.)

In [95]:

# create map of SEOUL using latitude and longitude values

latitude = seoul_subway_data['Lat'].mean()
longitude = seoul_subway_data['long'].mean()

map_seoul = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, statition_nm in zip(df_both['Lat'], df_both['long'], df_both['statition_nm']):
    label = '{}'.format(statition_nm)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seoul)  
    
for lat, lng, district_nm, foreign_resident_count_except_korean_chinese in zip(foreigner_data['Lat'], foreigner_data['long'], foreigner_data['en_nm'], foreigner_data['Foreign Resident']):
    label = '{}'.format(district_nm)
    label = folium.Popup(label, parse_html=True)
    radius = foreign_resident_count_except_korean_chinese/500.
    folium.CircleMarker(
        [lat, lng],
        radius=radius,
        popup=label,
        color='red',
        fill=True,
        fill_color='#6611cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seoul)      
    
map_seoul

## 3. Analyze Each Station Area
The Yongsan-gu area was chosen as the best place for the new restaurant through data analysis.

Reason
 *  3 selected subway Station 
 *  TOP 1 foregin resident counts

In order to analyze the existing commercial districts, we will use the Foursquare API to look at the current status of restaurants in question

In [103]:
#Focus Yongsan-gu Distriction


latitude = 37.531101
longitude = 126.981074

map_ys = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, statition_nm in zip(df_both['Lat'], df_both['long'], df_both['statition_nm']):
    label = '{}'.format(statition_nm)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ys)  
    
for lat, lng, district_nm, foreign_resident_count_except_korean_chinese in zip(foreigner_data['Lat'], foreigner_data['long'], foreigner_data['en_nm'], foreigner_data['Foreign Resident']):
    label = '{}'.format(district_nm)
    label = folium.Popup(label, parse_html=True)
    radius = foreign_resident_count_except_korean_chinese/500.
    folium.CircleMarker(
        [lat, lng],
        radius=radius,
        popup=label,
        color='red',
        fill=True,
        fill_color='#6611cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ys)      
    
map_ys


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [104]:
CLIENT_ID = 'ZE0EG4TU1SYNIBOI5CFK25XB32NV5I5CA0M0IV0UXEB41RAK' # your Foursquare ID
CLIENT_SECRET = 'J5FIFV0G0J4OLGEGLTTSV1IHQNQUXHLW1EJPX1SBWTXLMUDZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZE0EG4TU1SYNIBOI5CFK25XB32NV5I5CA0M0IV0UXEB41RAK
CLIENT_SECRET:J5FIFV0G0J4OLGEGLTTSV1IHQNQUXHLW1EJPX1SBWTXLMUDZ


In [137]:
##Itawon Sutway Station Area
df_both.head()
df_itaewon = df_both.loc[df_both['statition_nm']=='Itaewon']
                         
subway_latitude = df_both.loc[27, 'Lat']
subway_longitude = df_both.loc[27, 'long']
subway_name = df_both.loc[27, 'statition_nm']               


print('Latitude and longitude values of {} are {}, {}.'.format(subway_name, 
                                                               subway_latitude, 
                                                               subway_longitude))


Latitude and longitude values of Itaewon are 37.534488, 126.994302.


#### Now, let's get the top 100 venues that are in itaewon subway station within a radius of 1,000 meters.
First, let's create the GET request URL. Name your URL url.

In [138]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    subway_latitude, 
    subway_longitude, 
    radius, 
    LIMIT)

In [139]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d00b3e49fb6b7248c634cb0'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-58ea146a005a0f7562c22114-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1d3941735',
         'name': 'Vegetarian / Vegan Restaurant',
         'pluralName': 'Vegetarian / Vegan Restaurants',
         'primary': True,
         'shortName': 'Vegetarian / Vegan'}],
       'id': '58ea146a005a0f7562c22114',
       'location': {'address': '용산구 보광로 117',
        'cc': 'KR',
        'city': '서울특별시',
        'country': '대한민국',
        'distance': 137,
        'formattedAddress': ['용산구 보광로 117',
         '이태원1동',
         '서울특별시',
         '서울특별시',
         '04391',
         '대한민국'],
       

In [140]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [141]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Plant Cafe & Kitchen,Vegetarian / Vegan Restaurant,37.533256,126.994345
1,LINE FRIENDS (라인프렌즈),Gift Shop,37.534682,126.996118
2,Tongue & Groove Joint,BBQ Joint,37.533996,126.994457
3,Blacklist,Cocktail Bar,37.533900,126.992091
4,Maple Tree House (단풍나무집),Korean Restaurant,37.535030,126.993904


In [142]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Let's find out how many unique categories can be curated from all the returned venues

In [145]:
# one hot encoding
itaewon_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
itaewon_onehot['station'] = "Itaewon"

# move neighborhood column to the first column
fixed_columns = [itaewon_onehot.columns[-1]] + list(itaewon_onehot.columns[:-1])
itaewon_onehot = itaewon_onehot[fixed_columns]

itaewon_onehot.head()

,station,African Restaurant,American Restaurant,Art Gallery,BBQ Joint,Bakery,Bar,Beer Bar,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Burger Joint,Café,Caribbean Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,French Restaurant,Fried Chicken Joint,Gift Shop,Grocery Store,Gym / Fitness Center,Halal Restaurant,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jazz Club,Korean Restaurant,Lounge,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Pub,Record Shop,Sake Bar,Salad Place,Shoe Store,Speakeasy,Steakhouse,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,Itaewon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Itaewon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Itaewon,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Itaewon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Itaewon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows  by taking the mean of the frequency of occurrence of each category

In [146]:
itaewon_grouped = itaewon_onehot.groupby('station').mean().reset_index()
itaewon_grouped

,station,African Restaurant,American Restaurant,Art Gallery,BBQ Joint,Bakery,Bar,Beer Bar,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Burger Joint,Café,Caribbean Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,French Restaurant,Fried Chicken Joint,Gift Shop,Grocery Store,Gym / Fitness Center,Halal Restaurant,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jazz Club,Korean Restaurant,Lounge,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Pub,Record Shop,Sake Bar,Salad Place,Shoe Store,Speakeasy,Steakhouse,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,Itaewon,0.01,0.02,0.01,0.03,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.02,0.03,0.01,0.01,0.03,0.09,0.01,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.01,0.03,0.01,0.01,0.07,0.06,0.01,0.01,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.03,0.01,0.01,0.01


In [148]:
num_top_venues = 5

for station in itaewon_grouped['station']:
    print("----"+station+"----")
    temp = itaewon_grouped[itaewon_grouped['station'] == station].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Itaewon----
               venue  freq
0        Coffee Shop  0.09
1  Korean Restaurant  0.07
2             Lounge  0.06
3        Pizza Place  0.04
4                Bar  0.04




In [149]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for itaewon station.

In [152]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
itaewon_venues_sorted = pd.DataFrame(columns=columns)
itaewon_venues_sorted['station'] = itaewon_grouped['station']

for ind in np.arange(itaewon_grouped.shape[0]):
    itaewon_venues_sorted.iloc[ind, 1:] = return_most_common_venues(itaewon_grouped.iloc[ind, :], num_top_venues)

itaewon_venues_sorted.head()

,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Itaewon,Coffee Shop,Korean Restaurant,Lounge,Bar,Pizza Place,Café,Vegetarian / Vegan Restaurant,BBQ Joint,Indian Restaurant,Salad Place


## 4. Cluster Station Area

In [154]:
# set number of clusters
kclusters = 1

itaewon_grouped_clustering = itaewon_grouped.drop('station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(itaewon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0], dtype=int32)

In [158]:
itaewon_venues_sorted

,Cluster Labels,station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Itaewon,Coffee Shop,Korean Restaurant,Lounge,Bar,Pizza Place,Café,Vegetarian / Vegan Restaurant,BBQ Joint,Indian Restaurant,Salad Place


In [159]:
# add clustering labels
itaewon_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

itaewon_merged = df_itaewon

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
itaewon_merged = itaewon_merged.join(itaewon_venues_sorted.set_index('station'), on='statition_nm')


itaewon_merged.head() # check the last columns!

,statition_nm,Lat,long,ko_nm,line,Station_nm,IN_AM,OUT_AM,INOUT_R_AM,IN_LUNCH,OUT_LUNCH,INOUT_R_LUNCH,IN_DINNER,OUT_DINNER,INOUT_R_DINNER,OUT_AM_IN_DINNER_RATIO,IN_AM_OUT_DINNER_RATIO,INOUT_R_NIGHT,OUT_DINNER_IN_NIGHT_RATIO,IN_NIGHT,OUT_NIGHT,06-07 IN,06-07 OUT,07-08 IN,07-08 OUT,08-09 IN,08-09 OUT,09-10 IN,09-10 OUT,11-12 IN,11-12 OUT,12-13 IN,12-13 OUT,13-14 IN,13-14 OUT,16-17 IN,16-17 OUT,17-18 IN,17-18 OUT,18-19 IN,18-19 OUT,19-20 IN,19-20 OUT,20-21 IN,20-21 OUT,21-22 IN,21-22 OUT,22-23 IN,22-23 OUT,23-24 IN,23-24 OUT,00-01 IN,00-01 OUT,indicator_info,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Itaewon,37.534488,126.994302,이태원,6호선,이태원,65440,87168,75.073421,57194,77031,74.24803,152919,199603,76.611574,57.002727,32.785078,125.650175,160.130766,124650,99204,12956,7586,15709,12660,20712,38808,16063,28114,15243,21905,19967,26366,21984,28760,31004,35777,38394,45100,42986,60592,35725,56865,35814,37046,42231,32905,45667,29173,31927,26672,4825,10454,both,0,Coffee Shop,Korean Restaurant,Lounge,Bar,Pizza Place,Café,Vegetarian / Vegan Restaurant,BBQ Joint,Indian Restaurant,Salad Place


In [169]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(itaewon_merged['Lat'], itaewon_merged['long'], itaewon_merged['statition_nm'], itaewon_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=100,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Conclusion

#### The Yongsan-gu area was chosen as the best place for the new restaurant through data analysis.

*Reason

    - 3 selected subway Station
    - TOP 1 foregin resident counts

As result of using the Foursquare API to look at the current status of restaurants in question, Turkish Restaurant is not yet popluar nearby Itaewon Station Area.
But Itaewon Station is the best place for a new restaurant because it is a place where locals and foreigners come to enjoy food from various countries.

